#PROJECT

In [1]:
import json
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

/Users/aaronsalazar/LocalDocs/Bologna/ai_industry/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:


# Effettua la richiesta GET all'API TfL
url = "https://api.tfl.gov.uk/AccidentStats/2019"  # Cambia l'anno a seconda dei dati che vuoi
response = requests.get(url)

# Carica la risposta in formato JSON
data = response.json()

# Converte il JSON in un DataFrame di pandas
df = pd.json_normalize(data)

# Salva il DataFrame in formato CSV
# df.to_csv('incidenti_tfl_2019.csv', index=False)

In [5]:
df_vehicles = df.copy()

if df_vehicles['vehicles'].isnull().any():
    print("Warning: Missing or invalid data detected in 'vehicles'. Dropping null values.")
    df_vehicles = df_vehicles.dropna(subset=['vehicles'])

df_vehicles = df_vehicles.explode('vehicles')

df_vehicles.drop(columns=['lat', 'lon', 'date', 'location', 'severity', 'borough', 'casualties'], inplace=True)

df_vehicles_normalized = pd.json_normalize(df_vehicles['vehicles'].values)

if '$type' in df_vehicles_normalized.columns:
    df_vehicles_normalized.drop(columns=['$type'], inplace=True)

df_vehicles = df_vehicles.reset_index(drop=True).join(df_vehicles_normalized)

df_vehicles.drop(columns=['vehicles', '$type'], inplace=True)

df_vehicles.head()


,id,type
0,345979,Motorcycle_500cc_Plus
1,345979,Car
2,345980,Motorcycle_50_125cc
3,345980,Car
4,345980,Car
5,345981,Car
6,345982,Motorcycle_50_125cc
7,345982,Car
8,345983,Car
9,345983,Car


In [6]:
df_vehicles.head

<bound method NDFrame.head of            id                   type
0      345979  Motorcycle_500cc_Plus
1      345979                    Car
2      345980    Motorcycle_50_125cc
3      345980                    Car
4      345980                    Car
...       ...                    ...
91301  719872    Motorcycle_50_125cc
91302  719872                    Car
91303  719873                    Car
91304  719874                    Car
91305  719874                    Car

[91306 rows x 2 columns]>

In [22]:
categories = {
    "Motorcycle": [
        "Motorcycle_0_50cc", "Motorcycle_50_125cc", 
        "Motorcycle_125_500cc", "Motorcycle_500cc_Plus"
    ],
    "Car": [
        "Car", "Taxi", "Minibus", "LightGoodsVehicle"
    ],
    "Heavy_Vehicles": [
        "BusOrCoach", "HeavyGoodsVehicle", "MediumGoodsVehicle", 
        "AgriculturalVehicle"
    ],
    "Pedalcycle": ["PedalCycle"],
    "Other": ["OtherMotorVehicle", "RiddenHorse"]
}

# Map types to categories
category_map = {v: k for k, types in categories.items() for v in types}
df_vehicles['category'] = df_vehicles['type'].map(category_map)

# Count the number of vehicles in each category for each id
category_counts = df_vehicles.groupby(['id', 'category']).size().unstack(fill_value=0)
# Ensure all main categories are represented as columns
for category in categories.keys():
    if category not in category_counts.columns:
        category_counts[category] = 0

# Reset the index to prepare for further analysis
category_counts = category_counts.reset_index()

category_counts.keys()
category_counts.to_csv('../dataframes/vehicles_by_cat.csv', index=False)